In [1]:
# Set up the notebook to import modules from relative paths
import os, sys

#'/home/user/example/parent/child'
current_path = os.path.abspath('.')

#'/home/user/example/parent'
parent_path = os.path.dirname(current_path)

sys.path.append(parent_path)

In [3]:
from data_pipeline import ETL_Pipeline 

dp = ETL_Pipeline('/workspace/shared-data/')
transformed_df = dp.process('transactions-1.csv')
transformed_df.head()

,job_enc,Entertainment,Home,Misc,Shopping,txn_weekday,txn_month,part_of_day,normalized_amt,normalized_city_pop,normalized_age,normalized_distance_from_merchant,is_fraud
0,372,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.000137,0.001194,0.2000,0.630251,0
1,431,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.003670,0.000043,0.3250,0.094353,0
2,308,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.007569,0.001421,0.5375,0.276613,0
3,330,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.001520,0.000659,0.4750,0.392202,0
4,116,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.001415,0.000026,0.2250,0.602417,0


In [4]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns

from IPython.display import display, HTML

# Display Properties
from IPython.display import display, HTML
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


sns.set(style="ticks", color_codes=True)

In [5]:
df = pd.read_csv('/workspace/shared-data/transformed_data.csv')
df.head()

,job_enc,Entertainment,Home,Misc,Shopping,txn_weekday,txn_month,part_of_day,normalized_amt,normalized_city_pop,normalized_age,normalized_distance_from_merchant,is_fraud
0,372,0.000,0.000,1.000,0.000,2.000,0.000,0.000,0.000,0.001,0.200,0.630,0
1,431,0.000,0.000,0.000,1.000,2.000,0.000,0.000,0.004,0.000,0.325,0.094,0
2,308,1.000,0.000,0.000,0.000,2.000,0.000,0.000,0.008,0.001,0.537,0.277,0
3,330,1.000,0.000,0.000,0.000,2.000,0.000,0.000,0.002,0.001,0.475,0.392,0
4,116,0.000,0.000,1.000,0.000,2.000,0.000,0.000,0.001,0.000,0.225,0.602,0


In [6]:
from dataset import Fraud_Dataset

# Create a fraud dataset with 5 folds 
fd = Fraud_Dataset(df,'is_fraud')

In [7]:
from model import Fraud_Detector_Model
from metrics import Metrics

metrics = Metrics()

# Get the training, validation and test datasets
X_train, y_train = fd.get_training_dataset(0)
X_test, y_test = fd.get_testing_dataset(0)
X_val, y_val = fd.get_validation_dataset(0)

# List of classifiers
classifiers = ['RF', 'GB', 'AB']

# Initialize Metric Arrays 
accs, acc_bals, specificitys, sensitivitys, precs, recalls, f1s, roc_aucs, avg_precs = [], [], [], [], [], [], [], [], []

for classifier in classifiers:
    # Train and test the model
    model = Fraud_Detector_Model(classifier)
    
    if (classifier == 'AB'):
        model.train(X_train, y_train)
    else:
        model.train(X_train, y_train, X_val, y_val)
    
    # Obtain the metrics 
    acc, acc_bal, specificity, sensitivity, prec, recall, f1, roc_auc, avg_prec = model.test(X_test, y_test)
    
    # Collect the metrics 
    accs += [acc]
    acc_bals += [acc_bal]
    specificitys += [specificity]
    sensitivitys += [sensitivity]
    precs += [prec]
    recalls += [recall]
    f1s += [f1]
    roc_aucs += [roc_auc]
    avg_precs += [avg_prec]

# Generate the Report
metrics.generate_report(accs, acc_bals, specificitys, sensitivitys, precs, recalls, f1s, roc_aucs, avg_precs, classifiers, '/workspace/shared-data/hemdev-705.603Spring24/FraudDetection/results/model-results.txt')


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
